In [1]:
from exp.utils import *
from exp.models import *
from exp.losses import *
from tqdm.notebook import tqdm
from multiprocessing import Pool
from PIL import Image

import torch
import torch.nn as NN
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, transforms

from matplotlib.colors import LinearSegmentedColormap
from captum.attr import visualization as viz
from captum.attr import IntegratedGradients
from captum.attr import GradientShap
from captum.attr import DeepLift
from captum.attr import Occlusion
from captum.attr import NoiseTunnel
from captum.attr import visualization as viz
from captum.attr import Saliency

from ipywidgets import interact

In [2]:
picked_labels = ["Atelectasis", "Cardiomegaly", "Pneumonia"]

In [4]:
#for label in picked_labels:
for label in ["Atelectasis"]:
    print(f"Computing metrics for '{label}'")
    
    seed = 92
    model_name = f"DenseNet121_v2_{label}"
    print(model_name)
    s = 224
    image_size = (s, s)
    labels = get_labels()
    bs = 16
    device = get_device()
    
    sigmoid = NN.Sigmoid()
    model = load_model(model_name)
    model.eval();
    model = model.to(device)
    
    seed_everything(seed=seed)
    
    train_df, _, test_df = get_dataframes(include_labels=labels, 
                                             small=False)

    train_df = get_binary_df(label, train_df)
    test_df = get_binary_df(label, test_df)

    train_label = train_df[[label]].values
    neg_weights, pos_weights = compute_class_freqs(train_label)
    neg_weights, pos_weights = torch.Tensor(neg_weights), torch.Tensor(pos_weights)

    _, test_tfs = get_transforms(image_size=image_size)

    test_ds  = CRX8_Data(test_df , get_image_path(), label, image_size=image_size, transforms=test_tfs)

    test_dl  = DataLoader(test_ds,  batch_size=bs, shuffle=False)

    criterion = get_weighted_loss_with_logits(pos_weights.to(device), 
                                              neg_weights.to(device))

    history = validate(model, 
             criterion, 
             test_dl, 
             model_name,
             device=device)
    
    tmp = history
    tmp["loss"] = tmp["loss"].mean()
    tmp = {k:[v] for k, v in tmp.items()}
    model_metrics = pd.DataFrame(tmp.values(), index=tmp.keys(), columns=[model_name])
    save_metrics(model_metrics, model_name)
    
FERTIG()

Computing metrics for 'Atelectasis'
DenseNet121_v2_Atelectasis
Using the GPU!


/home/favi/work/crx8/exp/utils.py:179: UserWarning: Train-Val-Split currently with patient overlap!
  warnings.warn("Train-Val-Split currently with patient overlap!")



Val: Loss: 0.135, Acc: 0.704, AUROC: 0.588
Saved metrics to 'DenseNet121_v2_Atelectasis_metrics.csv'
FERTIG! :D


In [4]:
load_model_metrics(model_name)

,DenseNet121_v1_Atelectasis,DenseNet121_v1_Pneumonia,DenseNet121_v1_Cardiomegaly
loss,0.119989,0.024459,0.026605
acc,0.629512,0.420847,0.750625
auroc,0.722531,0.549110,0.845857
threshold,0.572989,0.504202,0.616187


In [9]:
load_metrics(model_name)

,DenseNet121_v2_Atelectasis
loss,0.135231
acc,0.704329
auroc,0.588102
threshold,0.541800
